In [8]:
import imdb
import requests
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

ia = imdb.IMDb()

In [9]:
# Otrzymanie comentarzów 
def get_imdb_reviews(movie_title):
    url = f"https://www.imdb.com/title/{movie_title}/reviews"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    review_containers = soup.find_all("div", class_="lister-item-content")

    reviews = []
    for container in review_containers:
        review_text = container.find("div", class_="text").text.strip()
        reviews.append(review_text)

    return reviews

In [11]:
import chardet
def load_data(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    data = pd.read_csv(file_path, encoding=encoding)
    reviews = data['comment'].tolist()
    labels = data['rating'].tolist()
    return reviews, labels

In [12]:
file_path = 'data.csv'
reviews, labels = load_data(file_path)

#for review, label in zip(reviews[:10], labels[:10]):
#    print("Rewiew: " + review)
#    print("Labels: " + str(label))

In [13]:
# Tworzenie i trening modelu
tokenizer = Tokenizer(num_words=10000)  # Ograniczenie słownictwa do 10 000 najczęściej używanych słów.
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
padded_sequences = pad_sequences(sequences, maxlen=100)  # Ogranicz długość sekwencji do 100 słów

In [14]:
model = None
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2023-06-02 19:21:26.295890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-02 19:21:26.552117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-02 19:21:26.552236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-02 19:21:26.559743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-02 19:21:26.559896: I tensorflow/compile

In [15]:
labels = np.array(labels)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, labels, epochs=7)

Epoch 1/7


2023-06-02 19:21:33.034942: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f9b15cff9e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-02 19:21:33.035036: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2023-06-02 19:21:33.101523: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-02 19:21:33.724416: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-02 19:21:34.201244: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-02 19:21:34.381480: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 7s 345ms/step - loss: 0.6920 - accuracy: 0.5533
Epoch 2/7
5/5 [==============================] - 2s 344ms/step - loss: 0.6624 - accuracy: 0.6933
Epoch 3/7
5/5 [==============================] - 2s 358ms/step - loss: 0.6387 - accuracy: 0.8667
Epoch 4/7
5/5 [==============================] - 2s 397ms/step - loss: 0.6138 - accuracy: 0.9533
Epoch 5/7
5/5 [==============================] - 1s 262ms/step - loss: 0.5843 - accuracy: 0.9667
Epoch 6/7
5/5 [==============================] - 2s 349ms/step - loss: 0.5492 - accuracy: 0.9800
Epoch 7/7
5/5 [==============================] - 1s 261ms/step - loss: 0.5042 - accuracy: 1.0000


In [16]:
# Funkcja analizy filmu
def analyze_movie(movie_id):
    reviews = get_imdb_reviews(movie_id)
    sequences = tokenizer.texts_to_sequences(reviews)
    padded_sequences = pad_sequences(sequences, maxlen=100)
    predictions = model.predict(padded_sequences)
    average_sentiment = predictions.mean()
    return average_sentiment

In [17]:
def get_imdb_movie_id(movie_title):
    movies = ia.search_movie(movie_title)
    movie = movies[0]
    movie_id = movie.getID()
    return 'tt' + movie_id

In [18]:
title = 'TOY STORY 2'
mv = get_imdb_movie_id(title)
#movie_title = get_imdb_movie_title_by_id(movie_id)
sentiment = analyze_movie(mv)
if sentiment > 0.5:
    print(f"The movie '{title}' is recommended.")
else:
    print(f"The movie '{title}' is not recommended.")

1/1 [==============================] - 0s 110ms/step
The movie 'TOY STORY 2' is not recommended.
